<a href="https://colab.research.google.com/github/neccher/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install py4j

     |████████████████████████████████| 204kB 5.2MB/s 


In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [ ]:
filter_total_votes_df = vine_df.filter("total_votes>20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filter_total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [ ]:
filter_helpful_votes_df = filter_total_votes_df.filter("helpful_votes/total_votes>.5").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filter_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [ ]:
filter_Y_vine_df = filter_helpful_votes_df.filter("vine=='Y'").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filter_Y_vine_df.summary().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|           103|               103|              103|               103| 103|              103|
|   mean|          null| 4.339805825242719|53.77669902912621|58.737864077669904|null|             null|
| stddev|          null|0.8111163701023958|44.55530426353715| 47.15493053567655|null|             null|
|    min|R118YQDTXS8FFY|                 2|               15|                21|   Y|                N|
|    25%|          null|                 4|               28|                33|null|             null|
|    50%|          null|                 5|               38|                42|null|             null|
|    75%|          null|                 5|               60|   

In [ ]:
filter_N_vine_df = filter_helpful_votes_df.filter("vine=='N'").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filter_N_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [27]:
from pyspark.sql.functions import count

paid_total = filter_Y_vine_df.count()
paid_5stars = filter_Y_vine_df.filter("star_rating==5").count()
paid_5stars_percentage = paid_5stars/paid_total
unpaid_total = filter_N_vine_df.count()
unpaid_5stars = filter_N_vine_df.filter("star_rating==5").count()
unpaid_5stars_percentage = unpaid_5stars/unpaid_total

In [28]:
paid_total

103

In [29]:
paid_5stars

55

In [31]:
paid_5stars_percentage * 100

53.398058252427184

In [32]:
unpaid_total

37372

In [33]:
unpaid_5stars

19723

In [34]:
unpaid_5stars_percentage * 100

52.77480466659531